In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName('MachineLearning') \
    .master('local[*]') \
    .getOrCreate()

In [2]:
input_path = '/home/jovyan/work/data/'
users_path = f'{input_path}cleaned_data_users/'

In [3]:
df = spark.read.csv(users_path, multiLine=True, header=True, inferSchema=True)

In [5]:
print(f'Dataframe rows number: {df.count()}')

Users-rows: 147577


In [6]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer

In [18]:
cols = ['_AccountId', 'forum', 'ActivityDays', 
        'comments_number', 'edits_of_answers', 'edits_of_question', 'created_questions', 
        'comments_every_day', 'activity_answer_edits', 'activity_question_edits', 'activity_questions']

In [19]:
df = df[cols]

In [20]:
df.printSchema()

root
 |-- _AccountId: integer (nullable = true)
 |-- forum: string (nullable = true)
 |-- ActivityDays: double (nullable = true)
 |-- comments_number: double (nullable = true)
 |-- edits_of_answers: double (nullable = true)
 |-- edits_of_question: double (nullable = true)
 |-- created_questions: double (nullable = true)
 |-- comments_every_day: double (nullable = true)
 |-- activity_answer_edits: double (nullable = true)
 |-- activity_question_edits: double (nullable = true)
 |-- activity_questions: double (nullable = true)



'OneHotEncoder'

In [22]:
str_cols = ['forum']
encoder = OneHotEncoder(inputCol='forum', outputCol='forum_features')

In [78]:
# unique list of forums
forums_list = df.select('forum').distinct().rdd.map(lambda r: r[0]).collect()

In [109]:
forum_udf = udf(lambda x: 1 if x == forum else 0)

In [127]:
for forum in forums_list:
    forum_udf = udf(lambda x: 1 if x == forum else 0)
    df = df.withColumn(f'forum_{forum}', forum_udf('forum'))
    df = df.withColumn(f'activity_days_{forum}', forum_udf('forum'))
    df = df.withColumn(f'comments_number_{forum}', forum_udf('forum'))
    df = df.withColumn(f'answer_edits_{forum}', forum_udf('forum'))
    df = df.withColumn(f'question_edits_{forum}', forum_udf('forum'))
    df = df.withColumn(f'created_questions_{forum}', forum_udf('forum'))
    df = df.withColumn(f'activity_comments_{forum}', forum_udf('forum'))
    df = df.withColumn(f'activity_answer_edits_{forum}', forum_udf('forum'))
    df = df.withColumn(f'activity_question_edits_{forum}', forum_udf('forum'))
    df = df.withColumn(f'activity_questions_{forum}', forum_udf('forum'))

In [128]:
exprs = {x: 'sum' for x in df.columns if x not in cols}

In [129]:
ddf=df.groupBy('_AccountId').agg(exprs)

In [132]:
# number of columns
len(ddf.columns)

71

In [25]:
ppl = Pipeline(stages=stage_string + stage_one_hot)
df = ppl.fit(df).transform(df)

In [37]:
from pyspark.sql.functions import *

Handling numerical data

In [28]:
from pyspark.ml.feature import Imputer

In [ ]:
num_cols = []

In [ ]:
# only for posts data
num_cols = ['_AnswerCount', '_CommentCount', '_ViewCount']
num_imputed = [var + "_imputed" for var in num_cols]

imputer = Imputer(inputCols = nume_cols, outputCols = num_imputed)
df = imputer.fit(df).transform(df)